# <strong>Firewalls</strong>
Firewalls are security features for networks designed to accept or reject connections to certain websites and ports. There are two types of firewalls: <strong>Stateless</strong> and <strong>Stateful</strong> firewalls.

<figure><center><img src="resources/firewall/firewall_graphic.jpg" style="width: 85%; height: 85%;"></img></center><em><figcaption>Credit: <a href="https://www.elegantthemes.com/blog/wordpress/what-is-a-firewall-and-which-type-is-right-for-you">Elegant Themes</a></figcaption></em></figure>

#### <strong>Stateless Firewalls</strong>
Stateless firewalls use rules to accept or block connections based on the source, destination, and other predefined criteria, without tracking the connection's state. The <u>state</u> of a connection refers to its status based on the packet sent. More about states will be covered when discussing stateful firewalls. Stateless firewalls tend to be quicker because they use predefined rules to filter information. Since they do not track states, they can be faster and less resource-intensive, making them effective for handling heavy traffic loads. Typically, small companies opt for stateless firewalls due to their affordability. <a href="https://www.fortinet.com/resources/cyberglossary/stateful-vs-stateless-firewall">Because there is usually less incoming traffic than with a large enterprise, there may also be fewer threats.</a>

#### <strong>Stateful Firewalls</strong>
Stateful firewalls are more intelligent because they can track and monitor active connections. There are three types of states recognized by ```iptables```, the tool you will use to configure firewalls in this lab:

|       State       | Description|
|:-----------------:|--------------------------------------|
| ```NEW```         | A new connection. Has not been seen in both directions (incoming and outgoing). |
| ```ESTABLISHED``` | A previously-seen connection. Has been seen at one point going in either direction. |
| ```RELATED```     | A different connection, but has been previously associated with a prior connection, <br>like FTP (File Transfer Protocol) or ICMP (Internet Control Message Protocol) errors. <br>These errors occur when the destination tells the source that a packet couldn't be <br>forwarded/delivered. |

Stateful firewalls are better at blocking a wider range of attacks, as many attacks can exploit a connection's state and sequences. These firewalls maintain a <strong>state table</strong>, where connections that are new, established, or related are tracked and updated as connections are made. These tables track the destination/source IP addresses and ports, as well as the connection's state.

For this lab, we are focusing on <u>stateful</u> firewalls.

<strong>This lab will cover four topics, and you will learn the following:</strong>

1. Basic Unix Tools for Networking
2. Introduction to ```iptables```
3. Basic ```iptables``` Rules
4. A Large-Scale Application of ```iptables```

In [19]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - umdsectc -c "/home/umdsectc/notebooks/resources/save.py pathname"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save():
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

def load_notebook():
    with load_lock:
        result = subprocess.run('su - umdsectc -c "/home/umdsectc/notebooks/resources/load.py pathname"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

### Step 0: Begin the experiment.

Click the button to begin creating the experiment.

In [2]:
# Click the button below to start the experiment.
def startlab(button):
    # Defining the lab name.
    labname = "pathnameumd"

    # Writing the information to an empty field below the button.
    with output0:
        output0.clear_output()
        
        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + ".umdsec[a-z]{1,2}"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - umdsectc -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing materialization for this lab already exists. </span><span>You might have ran another \
            lab without stopping this one. Attaching the existing materialization.</span>"))
            subprocess.run(f'su - umdsectc -c "mrg xdc attach xdc '+ match.group(0) + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing materializations are found.</span>"))
        
            # Second, start the lab.
            display(HTML("<span>Starting " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            startexp = subprocess.run('su - umdsectc -c "bash /share/startexp ' + labname + '"', capture_output=True, text=True, shell=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout)

            # Another lab is already attached to the XDC.
            if (("XDC already attached") in startexp.stdout):
                existingLab = re.search(r"real.(.*).umdsec[a-z]{1,2}", startexp.stdout).group(1)

                # Shouldn't happen.
                if (labname == existingLab):
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    os.popen('su - umdsectc -c "mrg xdc detach xdc.umdsectc"')
                    display(HTML("<span>Attaching the current lab.</span>"))
                    os.popen('su - umdsectc -c "mrg xdc attach xdc ' + materialPattern + '"')
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))
            runlab = subprocess.run('su - umdsectc -c "bash /home/runlab ' + labname + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))

# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/home/umdsectc/notebooks/saves/umdsectc_pathname.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 0):
                output0_2.clear_output()
                display(HTML("<span style='color: green;'>Your lab has been successfully loaded. Please click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to reflect your changes.</span>"))
            elif (result.returncode == 2):
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>The buffer node is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

## <strong>Topic 1: Basic Unix Tools for Networking</strong>

Before diving into firewalls, it's important to learn what tools are available to you for examining network connections. You will be provided a few tools that can assist you with checking the work that you make throughout the lab.

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Important:</strong></span> The firewall lab does NOT have a ```firewall``` node. There are two separate nodes for this lab, and you will be setting up different firewall rules on the ```server``` and ```client``` nodes. When you access your lab, use ```ssh server``` and ```ssh client```. <strong>SSH'ing to a ```firewall``` node will not work!</strong>

### Step 1: ```nmap``` - "<u>N</u>etwork <u>Map</u>ping"

```nmap``` is a useful tool that shows what ports that a website has access to your device. When using ```nmap```, you may call a website as an argument to see what ports that it uses on your machine. ```nmap``` has many additional uses, but for the purpose of this notebook, it will only be used to display the ports used by connections.

If you're interested in learning more about the uses of ```nmap```, you may check out some examples provided by the developers <a href="https://nmap.org/book/man-examples.html">here</a>.

<strong>Your task</strong>: Type a command below that uses ```nmap```, which displays the ports that <span style="color: blue;">yahoo.com</span> uses on your device. Do this in the ```server``` node.

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> The net tools that you are using in this lab may require ```sudo```. If you are getting a ```command not found``` error, you may either switch to sudo by using ```sudo su -```, or use ```sudo``` in front of your commands.

### Step 2: ```ifconfig``` - "<u>I</u>nter<u>f</u>ace <u>Config</u>uration"

```ifconfig``` is used to view and configure the network interfaces on your device. Network interfaces can be physical hardware components such as Wi-Fi adapters and Ethernet connections that send and receive data over the internet. Each type of connection, whether it's a Wi-Fi adapter, Ethernet connection, or even a virtual network interface, is treated as a separate interface by the operating system. 

For example, using a Wi-Fi dongle on a device that already has an internal network adapter will result in two different interfaces being displayed by ```ifconfig```. 

<strong>Your task</strong>: Use ```ifconfig``` in the ```server``` node. In the field below, type the name of the network interface whose IP address is ```10.0.1.1```.

### Step 3: ```telnet``` - "<u>Tel</u>e-<u>Net</u>work"

```telnet``` is a cleartext remote terminal protocol. On the surface, ```telnet``` is very simple: the user issues commands over a TCP socket, and the server replies with the results of those commands and waits for more input. ```telnet``` is one of the simplest and oldest network protocols still in use. Due to its cleartext nature and low-level access to the system, ```telnet``` is incredibly insecure. 

In the past, it was common for system administrators to log in as root using telnet on a network connection that could be easily sniffed by any sufficiently prepared attacker. This lack of encryption means that usernames, passwords, and other sensitive data are transmitted in plaintext, making it vulnerable to interception.

Telnetting to a suspected open port is still one of the fastest ways to see if a service is available or reachable. Here is an example of a ```telnet``` request.

```
$ telnet yahoo.com 80

Trying 66.94.234.13...
Connected to yahoo.com.
Escape character is '^]'.
GET /
...

<html><head> ...[web page data] ...
</body>
</html>

Connection closed by foreign host.
```

Telnetnetting to an IP and port (see above) should return a "connected" message if it is possible to connect to a running server.

<strong>Your task</strong>: Use ```telnet``` to retrieve <span style="color: blue;">https://scse.d.umn.edu/majors-minors</span>. Note the use of ```https``` within the URL. An ```https``` request uses port ```443```. Once you have a working connection, type the ```telnet``` command that you used, as well as the ```HTTP``` argument that you used.

### Step 4: ```nc``` - "<u>N</u>et<u>c</u>at"

```nc``` is a Unix utility for creating and using TCP and UDP sockets. In a very simplified way, ```nc``` is like a telnet client and server without any built in protocol or terminal emulation. Another way of putting it is that netcat is the bare essentials for creating a TCP or UDP socket and client, with hooks for using standard in and standard out for IO.

Netcat can be used for several purposes, such as: Sending messages to computers, debugging connection issues, scanning for open ports on a remote host (```nc -zv hostname 20-80``` scans between ports 20-80 on the specified hostname), transferring files, creating a backdoor (which bypasses security systems), connecting to a remote shell, and a lot more.

<strong>Your task</strong>: On the ```server``` node, set up a server that listens for UDP protocols, and listen on port 10000. On the ```client``` node, set up a connection to the ```server``` node that sends UDP packets to ```server``` on port 10000. Once you have set this up, try sending a message through ```client``` to see if it will be displayed on ```server```.

There are text entries below, where you will type in the commands that you used to set up ```server``` and ```client```. Once you have a working "chatroom", type the two commands below that worked. They will be tested.

## <strong>Topic 2: Introduction to ```iptables```</strong>

In the previous section, you have explored some tools that you may find useful with creating firewall rules in this lab. Now, you're going to look at the next Unix tool, which is ```iptables```. You will be spending the remaining part of the lab learning about ```iptables```. Towards the end, you will be given a list of rules, one at a time, and implementing them into your nodes.

This section will walk you through the basic interface of ```iptables```.

### Step 5: Viewing ```iptable``` Rules

The ```iptables``` tool has been pre-installed for you on both nodes. However, you will need to navigate into ```server``` for the remainder of this topic.

Type ```sudo iptables -L``` into ```server```. ```sudo``` is required for this tool, since this tool lets you control the network flow across the computer. The ```-L``` argument stands for "list". Typing this command is going to list all of the rules that are currently in your rule table.

These are the only three policies at the moment:
```
-P INPUT ACCEPT
-P FORWARD ACCEPT
-P OUTPUT ACCEPT
```

Before providing a breakdown of these three policies, you are going to hear the term <strong>"chain"</strong> when working with ```iptables```. A chain is a set of rules. Hence, you're "linking" the rules together to form a chain.

Here's a small breakdown of the default rules so far:
- The ```-P``` argument means "policy". This parameter is used with three built-in chains: ```INPUT```, ```FORWARD```, or ```OUTPUT```. There are more built-in chains than these, but these are the most common ones when using this argument.
  - Note, ```-p``` stands for "protocol". These two should not be mixed up when you begin to make your own rules. You can use either ```-P```, or to be more precise, ```--policy``` also works.
- ```INPUT```, ```FORWARD```, and ```OUTPUT``` are the incoming, forwarded, and outgoing packets.
- ```ACCEPT``` is the policy of the chain. It tells your system what to do with the chain that preceeds it.
  - <strong>All ```iptables``` rules end with ```ACCEPT```, ```DROP```, or ```REJECT```.</strong>


After reading this breakdown, this should make the default rules clear in ```iptables```. This means that all incoming, forwarded, and outgoing packets are accepted. Nothing is being blocked.

<strong>Your task</strong>: Add the following rule to the table. You will learn about what it does in the next two steps.

```sudo iptables -A OUTPUT -p tcp --dport 80 -j DROP```

You may view this rule with ```sudo iptables -L```.

### Step 6: Identifying the Chain and Jump

With this rule added, it's important to understand what this rule has done.

Here is a breakdown of the rule that you just added:
- ```sudo iptables``` calls the ```iptables``` command.
- ```-A OUTPUT``` <strong>appends</strong> the rules that you made to the built-in ```OUTPUT``` chain.
- ```-p tcp``` indicates that the rule affects TCP <strong>packets</strong>.
- ```--dport 80``` indicates that the rule affects the <strong>destination port</strong>
- ```-j DROP``` stands for <strong>jumps</strong>. However, it's easier to interpret this as "everything that matches this rule will be DROPPED".

The reason why ```-A``` is being used is because you are taking the currently existing ```-P OUTPUT ACCEPT``` rule, but you're saying, "Accept all output, BUT... (your rule here)".

Using the two blanks below, type in the <strong>chain</strong> and <strong>jump</strong> of the rule that you wrote.

### Step 7: Testing Your Rule

Using one of the tools that you learned in Topic 1, type a command that will attempt to access a website through port 80. Do not use a command that shows that port 80 is unavailable. You will need to type a command that attempts to access the website.

### Step 8: Deleting a Rule

To delete a rule, you can use ```sudo iptables -D [rule]```. Clearly, the ```-D``` stands for delete. Previously, you used ```-A``` to append a rule.

Using your intuition, delete the rule that you made in Step 5.

## <strong>Topic 3: Basic ```iptables``` Rules <strong>

This topic is now going to teach you some ```iptables``` syntax. You have already seen a little bit of syntax in the previous step. Now, you're going to look more in-depth with ```iptables``` syntax.

```iptables``` allows you to create very broad commands, like the one you have seen above. Additionally, it allows you to create very specific commands, like you're about to see in this topic. The following steps are going to start you with a very basic example, and you are going to work your way up to specifics.

<u>As you complete each step in this topic</u>, a table will be generated so that you can see how your rule is being progressively created. The table will provide a "translation", which is describing how your rules are being defined.

If you are feeling lost with ```iptables``` syntax, these are two helpful documents that you may look into while you go through the final two topics:
- <a href="https://linux.die.net/man/8/iptables">```iptables``` Documentation</a>
- <a href="https://www.informit.com/articles/article.aspx?p=421057&seqNum=4">Linux Firewalls, 3rd Edition - Iptables Chapter</a>

### Step 9: Tables

This is the start of the ```iptables``` rule that you'll be producing throughout the topic:

```sudo iptables```

<strong>You are going to construct a rule that applies to the ```server``` node. NOT the ```client``` node.</strong>

There are multiple tables that you can add rules to in ```iptables```. For the lab, you're just adding tables to the "filter" table. This is the default table in ```iptables```. The remaining tables are as follows - <a href="https://gist.github.com/nerdalert/a1687ae4da1cc44a437d">(source)</a>:
- NAT table - Iptable's NAT table has the following built-in chains.
- Mangle table - Iptables's Mangle table is for specialized packet alteration.
- Raw table - Iptable's Raw table is for configuration excemptions.

You are going to append to the currently existing command. This is the current template of your rule so far:

```sudo iptables [your answer here]```

<strong>Your task</strong>: Type a parameter that adds your rule to the "filters" table. Use the ```-t``` argument for "table". For your answer, just type in the ```[your answer here]``` portion of the rule.

### Step 10: Network Interface

Now, you can add a network interface that you want your rule to be applied to. You can do this by using ```-i``` for "interface". From Step 2, you already found what the network interface is attached to ```server```.

Here is the work so far that you've produced:

```sudo iptables [step 9] [your next answer]```

<strong>Your task</strong>: Type in the following parameters which will match the following rules:

- Use the network interface that you found in Step 2.
- Append your rule to the pre-defined OUTPUT chain.

### Step 11: States and Protocols

Next, you're going to add a state to the rule. This is done by using the ```-m``` and ```--state``` command. These two parameters are used in conjunction with each other. ```-m``` stands for "match", and ```--state``` is clearly indicating the state to match to.

Recall states from the beginning of the notebook. There are ```NEW```, ```ESTABLISHED```, and ```RELATED``` states. This is the template for creating a state rule: ```-m state --state STATE_NAME```

A breakdown:
- ```-m``` means match.
- ```state``` means you're going to match a state.
- ```--state``` indicates that the next parameter is going to be a state.
- ```STATE_NAME``` is either ```NEW```, ```ESTABLISHED```, or ```RELATED```.

Additionally, you read above that ```-p``` means "protocol". Two common protocols are UDP and TCP protocols. If you haven't taken a networking course, you may read about the two protocols <a href="https://stackoverflow.com/questions/5970383/difference-between-tcp-and-udp">here</a>. Essentially, <strong>UDP</strong> is <strong>U</strong>nreliable, but fast. <strong>TCP</strong> is <strong>T</strong>rustworthy, but slow.

Here's the rule so far:

```sudo iptables [step 9] [step 10] [your next answer]```

Clearly, you should see how the remaining steps in this topic will work. Additionally, your answer is being printed as you work along.

<strong>Your task</strong>: Type in the following parameters which will match the following rules:

- Create a rule that matches the NEW state.
- Detect TCP packets.

### Step 12: Sources and Destinations

For this step, you're going to apply a source and destination to this rule. These two parameters are fairly straightforward. You will use ```-s``` for the source IP address, and ```-d``` for the destination IP address. Additionally, you can use ```--sport``` or ```--dport``` for the source/destination ports of the rule.

<strong>Your task</strong>: Type in the following parameters which will match the following rules:

- Use the source of the network interface that you found in Step 2.
- Use the destination of ```google.com```.
  - Hint: Use one of the tools from Topic 1 to find the IP address of Google.
- Add a rule that uses Google's destination port of 1234.

### Step 13: Jumps

Finally, you are going to do the jump. This indicates what you are going to do with packets that match all of these rules that you composed so far. You have already seen how a jump works. Use the ```-j``` command. Recall that all ```iptables``` rules will end with a jump.

<strong>Your task</strong>: Drop all connections that match the rule that you have been constructing in this topic.

Your final rule will be printed upon completing this step.

Final rule:

```sudo iptables -t filter -A OUTPUT -i eth1 -p tcp -s 10.0.1.1 -d 192.168.1.100 --dport 1234 -m state --state NEW -j DROP```

## <strong>Topic 4: A Large-Scale Application of ```iptables```<strong>

In this final topic, you are going to be given an instruction to block or accept certain connections. Using all of the knowledge you used in the previous topics, you are prepared to attempt to create your own rules.

Once again, as provided in the previous topic, you're free to use these two resources if you feel stuck with the syntax for ```iptables```:

- <a href="https://linux.die.net/man/8/iptables">```iptables``` Documentation</a>
- <a href="https://www.informit.com/articles/article.aspx?p=421057&seqNum=4">Linux Firewalls, 3rd Edition - Iptables Chapter</a>

Additionally, these are the following IP addresses that you will need to use for this section:

| Nodes:        | ```server```   | ```client```   |
|---------------|----------------|----------------|
| IP Addresses: | ```10.0.1.1``` | ```10.0.1.2``` |

#### <strong>Useful Information For This Section</strong>:

- Whenever you would like to reset your ```iptables``` rules, you may type ```sudo iptables -F```. The ```-F``` command means "flush", which will reset your table back to what it originally was. If you had already completed a step, your progress will not be lost after flushing the table.

- Additionally, <u>you will need to indicate a specific protocol in each of your steps</u>. In other words, all of your rules must have ```-p udp``` or ```-p tcp``` in them. By default, most connections that are made between ```client``` and ```server``` are TCP connections. However, the last step will require a UDP connection.

- Use ```sudo iptables -S``` to view all current rules.

### Step 14: Inbound TCP Connections to the Apache Server

As you do the questions provided in this topic, you will be provided a "Test Connection" button. These checks will take time to complete, since it's testing for time-outs. Additionally, you are going to be told which node that these firewall rules will need to be placed on.

<strong>Scenario:</strong> On the ```server``` node, <strong>disallow</strong> inbound TCP connections to the Apache port from ```client```. This port is ```80```. Currently, this port is open to all connections to the ```server``` node.

<strong>A successful test appears as follows:</strong>
```
umdsecXX@client:~$ telnet server 80
Trying 10.0.1.1...
^C
umdsecXX@client:~$
```

Click the button below to check your work.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> You will need to use ```client```'s IP address in your rule.

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> You are now starting to apply firewall rules, which means that an incorrect firewall rule could potentially lock you out from your node.

If you get locked out, you will have to end your lab (at the bottom of the notebook), then restart your lab by clicking "Start Lab" at the top of the notebook.

Your progress is automatically saved at each step, so you do not need to worry about losing progress.

### Step 15: Inbound TCP Connections to the MySQL Server

<strong>Scenario:</strong> On the ```server``` node, <strong>disallow</strong> inbound TCP connections to the MySQL port from ```client```. This port is ```3306```. Currently, this port is open to all connections to the ```server``` node.

<strong>A successful test appears as follows:</strong>
```
umdsecXX@client:~$ telnet server 3306
Trying 10.0.1.1...
^C
umdsecXX@client:~$
```

Click the button below to check your work.

### Step 16: Outbound TCP Connections to a Specific HTTPS Request

<strong>Scenario:</strong> On the ```client``` node, <strong>allow</strong> outbound HTTPS connections to <span style="color: blue;">stackoverflow.com</span>. The port for HTTPS requests is ```3306```. Currently, all outbound HTTPS requests are blocked on ```client```. They all must remain blocked, unless it's Stack Overflow.

<u>You will need to find the IP address of Stack Overflow.</u>

<strong>A successful test appears as follows:</strong>
```
umdsecXX@client:~$ telnet stackoverflow.com 443
Trying XXX.XX.XX.X...
Trying XXX.XX.XXX.XXX...
Connected to stackoverflow.com.
Escape character is '^]'.
Connection closed by foreign host.
umdsecXX@client:~$
```

Click the button below to check your work. 

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Note:</strong></span> In order for this test to pass, other random websites will be accessed. Other websites must not be accessible.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> <strong>Two rules are required for this step.</strong> Rules on ```iptables``` go in order. If you attempt to block all outgoing connections first, then allow Stack Overflow through the firewall, it will not work. You must allow Stack Overflow first, then block the remaining connections.

```
-P INPUT ACCEPT
-P FORWARD ACCEPT
-P OUTPUT ACCEPT
-A OUTPUT -d 172.64.155.249/32 -p tcp -m tcp --dport 443 -j ACCEPT
-A OUTPUT -p tcp -m tcp --dport 443 -j REJECT --reject-with icmp-port-unreachable
-A OUTPUT -p udp -m udp --dport 53 -j ACCEPT
-A OUTPUT -p tcp -m tcp --dport 53 -j ACCEPT
```

### Step 17: Outbound TCP OpenSSH Requests

<strong>Scenario:</strong> On the ```client``` node, <strong>disallow</strong> outgoing SSH requests. Using OpenSSH uses port 22.

<strong>A successful test appears as follows:</strong>
```
umdsecXX@client:~$ ssh server
ssh: connect to host server port 22: Connection refused
umdsecXX@client:~$
```

Click the button below to check your work. 

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> Double-check that you are going to deny any <strong>outgoing</strong> SSH requests. If you accidentally deny any incoming SSH requests, you may lose access to your ```client``` node, and the lab must be restarted.

```
-P INPUT ACCEPT
-P FORWARD ACCEPT
-P OUTPUT ACCEPT
-A OUTPUT -p tcp -m tcp --dport 22 -j REJECT --reject-with icmp-port-unreachable
```

### Step 18: Inbound UDP Traffic on Ports 10000:10005

<strong>Scenario:</strong> On the ```server``` node, <strong>allow</strong> UDP connections to be made on six ports: ```10000```, ```10001```, ```10002```, ```10003```, ```10004```, and ```10005```. You do not need to apply rules on the ```client``` node. Just the ```server``` node.

<u>You are required to block UDP connections on any other port.</u> This test will attempt to try accessing other ports on ```server``` for UDP connections.

<strong>A successful test appears as follows:</strong>

<u>Part 1:</u>
```
umdsecXX@server:~$ nc -u -l 10000
Hello!

umdsecXX@client:~$ nc -u server 10000
(Type Hello!)
```

<u>Part 2:</u>

```
umdsecXX@server:~$ nc -u -l 10006


umdsecXX@client:~$ nc -u server 10006
(Type Hello!)
```

Click the button below to check your work. 

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> You do not need to make six independent rules for the ports. You can just call 10000:10005.

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> ```server``` relies on some UDP connections for ```iptables``` to run properly. Therefore, make sure to add the source IP address for ```client```, which is provided at the start of the topic.

```
sudo iptables -A INPUT -p udp -s 10.0.1.1 --dport 10000:10005 -j ACCEPT
sudo iptables -A INPUT -p udp --dport 10000:10005 -j ACCEPT
sudo iptables -A INPUT -p udp -j DROP
```

## <strong>Grading</strong>

To check your overall grade, click on the button below.

<strong>*Warning:* Your submission will be ran on an unmodified copy of this notebook</strong>. If you tinkered with any of the input cells to pass a step, you will not get full credit.

### Stopping the Lab

Once you are done with the lab, click on the "Stop Lab" button below. <strong>This will delete your materialization, which will delete all of the lab's resources.</strong> Your progress is saved automatically in ```~/notebooks/submissions```. This folder is stored on your XDC, and will not be deleted upon stopping the lab.

IMPORTANT INFORMATION FOR INSTALLING THE LAB.

Download net-tools and run ```ifconfig``` to get the eth0 IP address. When Topic 3 is complete, add this rule:


```sudo iptables -A INPUT -m conntrack --ctstate ESTABLISHED,RELATED -j ACCEPT``` - Make existing/related connections allowed.
```sudo iptables -A INPUT -p tcp --dport 22 -s client_ip -j ACCEPT``` - Allow all connections from the student's XDC.

```sudo iptables -A INPUT -p tcp --dport 22 -s 10.1.14.49 -j ACCEPT```

